In [ ]:
!nvcc --version
%pip install nvcc4jupyter
%load_ext nvcc4jupyter

In [14]:
%%cuda
#include<stdio.h>
#include<stdlib.h>
#define N 20
#define BLOCK_SIZE 256

__global__
void addArray(int* arr,int* sumOutput){
    __shared__ 
    int sharedSum[BLOCK_SIZE];

    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + tid;

    if(i<N){
        sharedSum[tid] = arr[i];
    }
    else{
        sharedSum[tid] = 0;
    }
    __syncthreads();

    for(int stride = blockDim.x/2; stride > 0 ; stride >>= 1 ){
        if(tid<stride){
            sharedSum[tid] = sharedSum[tid] + sharedSum[tid+stride];
        }
        __syncthreads();
    }

    if( tid == 0){
        sumOutput[blockIdx.x] = sharedSum[0];
    }
}

int main(){
    size_t sz = N*sizeof(int);
    int numberOfBlocks = (N+BLOCK_SIZE-1)/BLOCK_SIZE;
    size_t szB = numberOfBlocks*sizeof(int);

    int* h_array=(int*)malloc(sz);
    int* h_blockSum=(int*)malloc(szB);
    int finalSum=0;

    for (int i = 0; i < N; i++) {
        h_array[i] = i;
        printf("%d ",h_array[i]);
    }
    printf("\n");

    int *d_array,*d_blockSum;

    cudaMalloc((void**)&d_array,sz);
    cudaMalloc((void**)&d_blockSum,szB);

    cudaMemcpy(d_array,h_array,sz,cudaMemcpyHostToDevice);

    addArray<<<numberOfBlocks,BLOCK_SIZE>>>(d_array,d_blockSum);

    cudaMemcpy(h_blockSum,d_blockSum,szB,cudaMemcpyDeviceToHost);

    for(int i=0;i<numberOfBlocks;i++){
        finalSum=finalSum+h_blockSum[i];
    }

    printf("Sum: %d\n",finalSum);

    cudaFree(d_array);
    cudaFree(d_blockSum);
    free(h_array);
    free(h_blockSum);

    return 0;
}


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
Sum: 190

